In [1]:
!pip install pydub language-tool-python librosa transformers

In [2]:
!pip install pytube ffmpeg openai

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.0 MB/s eta 0:00:00
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=27c62f863ea10ae69cdd57f378c0ba85270d1b9d358d94a537dec13d464c9840
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [3]:
!pip install google-cloud-speech

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.0/305.0 kB 9.2 MB/s eta 0:00:00


In [1]:
import os
import io
from google.cloud import speech

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=""

In [3]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

pluginlive-444306-d4f6569e136a.json


In [4]:
!pip install eng-to-ipa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 34.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eng-to-ipa: filename=eng_to_ipa-0.0.2-py3-none-any.whl size=2822601 sha256=a18521cd0397f5aa045224978f6bd7b0fca850b612771e4b3d4b22ac418bf1cb
  Stored in directory: /root/.cache/pip/wheels/5b/ab/07/fe6722f710d8ef8bd0ccb4eb689ef96f5552f3fc0c80c1aa9c
Successfully built eng-to-ipa


In [5]:
import google.generativeai as genai

genai.configure(api_key="")
language_model = genai.GenerativeModel("gemini-pro")

In [9]:
import os
import torch
from pydub import AudioSegment, silence
import language_tool_python as lt
import librosa
import numpy as np
from openai import OpenAI
import string
import eng_to_ipa as ipa
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torchaudio

input_audio_path = "/content/sample.wav"
temp_audio_path = "resampled.wav"

import soundfile as sf

def preprocess_audio(file_path, target_sr=16000):
    y, sr = librosa.load(file_path, sr=None)
    if sr != target_sr:
        y = librosa.resample(y, orig_sr=sr, target_sr=target_sr)
    sf.write(temp_audio_path, y, target_sr, subtype="PCM_16")
    return temp_audio_path, target_sr

def transcribe_audio(audio_path):

    client = speech.SpeechClient()


    with open(audio_path, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)


    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        language_code="en-US",
        enable_automatic_punctuation=True,
    )


    response = client.recognize(config=config, audio=audio)


    transcription = " ".join([result.alternatives[0].transcript for result in response.results])
    return transcription
def grammar_analysis(text):
    prompt = f"""
    You are an English grammar expert. You will be provided with the response of a candidate.
    Your task is to parse the response and generate the number of grammatical errors, evaluating the
    candidate's grammatical correctness. Avoid considering punctuation marks because this is transcribed.
    Candidate's Response:
    {text}
    Please provide the number of grammatical errors without any explanation.
    """

    response = language_model.generate_content(prompt)
    grammar_errors = response.text
    return int(grammar_errors)
def text_to_phonemes(text):
    phonemes = ipa.convert(text)
    return phonemes.split()
def pronunciation_analysis(transcribed_text, original_audio_path):
    word_count = len(transcribed_text.split())
    audio_duration = librosa.get_duration(filename=original_audio_path)
    speaking_rate = word_count / audio_duration
    return word_count, audio_duration, speaking_rate

def fluency_analysis(transcribed_text, audio_path):
    audio = AudioSegment.from_file(audio_path)
    silences = silence.detect_silence(audio, min_silence_len=2000, silence_thresh=-40)
    print(silences)
    num_pauses = len(silences)
    filler_words = ["um", "uh", "like", "so", "actually", "basically", "literally", "you know"]
    words = transcribed_text.translate(str.maketrans('', '', string.punctuation)).split()
    detected_fillers = []
    for i in range(len(words)):
        if words[i].lower() in filler_words:
            detected_fillers.append(words[i])
        if i < len(words) - 1 and f"{words[i].lower()} {words[i+1].lower()}" == "you know":
            detected_fillers.append("you know")
    return num_pauses, detected_fillers

def phonetic_analysis(transcribed_text, audio_path):
    expected_phonemes = text_to_phonemes(transcribed_text)
    processor = Wav2Vec2Processor.from_pretrained("Bluecast/wav2vec2-Phoneme")
    model = Wav2Vec2ForCTC.from_pretrained("Bluecast/wav2vec2-Phoneme")
    audio, rate = torchaudio.load("/content/sample.wav", normalize=True, channels_first=True)
    if audio.shape[0] == 2:
        audio = audio.mean(dim=0, keepdim=True)
    target_rate = 16000
    if rate != target_rate:
        transform = torchaudio.transforms.Resample(orig_freq=rate, new_freq=target_rate)
        audio = transform(audio)
        rate = target_rate
    audio = audio.squeeze(0)
    inputs = processor(audio, sampling_rate=rate, return_tensors="pt", padding=True)
    logits = model(inputs.input_values).logits
    pred_ids = logits.argmax(axis=-1)
    phonemes = processor.batch_decode(pred_ids)
    if isinstance(phonemes, list):
        phonemes = phonemes[0].split()
    phoneme_to_ipa = {
    'aa': 'ɑ',   # Father
    'ae': 'æ',   # Cat
    'ah': 'ʌ',   # Cup
    'ao': 'ɔ',   # Thought
    'aw': 'aʊ',  # House
    'ax': 'ə',   # About
    'ax-h': 'ə', # Ahead (unstressed)
    'axr': 'ɚ',  # Butter (unstressed)
    'ay': 'aɪ',  # My
    'eh': 'ɛ',   # Bed
    'el': 'l̩',  # Syllabic 'l' (Bottle)
    'em': 'm̩',  # Syllabic 'm' (Hymn)
    'en': 'n̩',  # Syllabic 'n' (Button)
    'er': 'ɝ',   # Bird (stressed)
    'ey': 'eɪ',  # They
    'ih': 'ɪ',   # Sit
    'iy': 'i',   # See
    'ow': 'oʊ',  # Go
    'oy': 'ɔɪ',  # Boy
    'uh': 'ʊ',   # Put
    'uw': 'u',   # Blue
    'ux': 'ʉ',   # Goose (Northern American English)
    'b': 'b',    # Bat
    'ch': 'tʃ',  # Cheese
    'd': 'd',    # Dog
    'dh': 'ð',   # This
    'dx': 'ɾ',   # Flap 't' (Butter)
    'eh': 'ɛ',   # Bed
    'f': 'f',    # Fish
    'g': 'g',    # Go
    'hh': 'h',   # Hat
    'jh': 'dʒ',  # Jump
    'k': 'k',    # Cat
    'l': 'l',    # Leg
    'm': 'm',    # Man
    'n': 'n',    # Net
    'ng': 'ŋ',   # Sing
    'p': 'p',    # Pet
    'q': 'ʔ',    # Glottal Stop (Uh-oh)
    'r': 'ɹ',    # Red
    's': 's',    # Sun
    'sh': 'ʃ',   # Shoe
    't': 't',    # Top
    'th': 'θ',   # Thing
    'v': 'v',    # Van
    'w': 'w',    # Wet
    'y': 'j',    # Yes
    'z': 'z',    # Zoo
    'zh': 'ʒ',   # Measure
    }
    ipa_transcription = [phoneme_to_ipa.get(phoneme, phoneme) for phoneme in phonemes]
    return ' '.join(ipa_transcription), expected_phonemes




print("Preprocessing audio...")
processed_path, sr = preprocess_audio(input_audio_path)

print("Transcribing audio...")
transcription = transcribe_audio(processed_path)
print(f"Transcription: {transcription}")

print("Evaluating grammar...")
grammar_errors = grammar_analysis(transcription)
print(f"Grammar Errors: {grammar_errors}")

print("Evaluating pronunciation...")
speaker_phonemes, expected_phonemes= phonetic_analysis(transcription, processed_path)
print(f"Phonemes: {expected_phonemes}")

print("Evaluating fluency...")
num_pauses, filler_words = fluency_analysis(transcription, processed_path)
word_count, duration, speaking_rate = pronunciation_analysis(transcription, processed_path)
print(f"Word Count: {word_count}")
print(f"Audio Duration: {duration:.2f} seconds")
print(f"Speaking Rate: {speaking_rate:.2f} words per second")
print(f"Number of Pauses: {num_pauses}")
print(f"Filler Words Detected: {len(filler_words)}")

print("\n### Final Assessment ###")
print(f"Transcription: {transcription}")
print(f"Grammar Errors:{grammar_errors}")
print(f"Word Count: {word_count}, Speaking Rate: {speaking_rate:.2f} WPS")
print(f"Number of Pauses: {num_pauses}")
print(f"Phonemes: {expected_phonemes}")
print(f"Filler Words Detected: {filler_words}")


if os.path.exists(temp_audio_path):
    os.remove(temp_audio_path)



Preprocessing audio...
Transcribing audio...
Transcription: Hey guys, how are you doing?
Evaluating grammar...
Grammar Errors: 1
Evaluating pronunciation...


model.safetensors:  92%|#########2| 1.16G/1.26G [00:00<?, ?B/s]

Phonemes: ['heɪ', 'gaɪz,', 'haʊ', 'ər', 'ju', 'duɪŋ?']
Evaluating fluency...
[]
Word Count: 6
Audio Duration: 4.71 seconds
Speaking Rate: 1.27 words per second
Number of Pauses: 0
Filler Words Detected: 0

### Final Assessment ###
Transcription: Hey guys, how are you doing?
Grammar Errors:1
Word Count: 6, Speaking Rate: 1.27 WPS
Number of Pauses: 0
Phonemes: ['heɪ', 'gaɪz,', 'haʊ', 'ər', 'ju', 'duɪŋ?']
Filler Words Detected: []


<ipython-input-9-ac9ed9556c26>:66: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  audio_duration = librosa.get_duration(filename=original_audio_path)


In [17]:
phonemes_speaker = speaker_phonemes.replace(" ", "")
print(phonemes_speaker)


ʌgaɪðhaʊɑɹjuduɪŋ


In [18]:
phonemes_without_punctuation = [''.join(ch for ch in phoneme if ch not in string.punctuation) for phoneme in expected_phonemes]
phonemes_true = ''.join(phonemes_without_punctuation)
print(phonemes_true)

heɪgaɪzhaʊərjuduɪŋ


In [27]:
import Levenshtein

def scoring_metrics(transcription, grammar_errors, num_pauses, filler_words, true_phonemes, speaker_phonemes):
  words= transcription.split(' ')
  num_words=len(words)
  grammar_mistake = 1- (grammar_errors/num_words)
  grammar_score= 5*grammar_mistake
  grammar_score=round(grammar_score,2)
  fluency_mistake = 1- ((num_pauses+len(filler_words))/num_words)
  fluency_score=5*fluency_mistake
  fluency_score=round(fluency_score, 2)
  len_s1, len_s2 = len(true_phonemes), len(speaker_phonemes)
  dp = [[0] * (len_s2 + 1) for _ in range(len_s1 + 1)]
  for i in range(len_s1 + 1):
      dp[i][0] = i
  for j in range(len_s2 + 1):
      dp[0][j] = j
  for i in range(1, len_s1 + 1):
      for j in range(1, len_s2 + 1):
          cost = 0 if true_phonemes[i - 1] == speaker_phonemes[j - 1] else 1
          dp[i][j] = min(dp[i - 1][j] + 1, dp[i][j - 1] + 1, dp[i - 1][j - 1] + cost)
  edit_distance = dp[len_s1][len_s2]
  max_length = max(len(expected_phonemes), len(speaker_phonemes))
  pronunciation_score = ((max_length - edit_distance) / max_length) * 5
  pronunciation_score=round(pronunciation_score, 2)
  return grammar_score, fluency_score, pronunciation_score+0.5

grammar_score, fluency_score, pronunciation_score=scoring_metrics(transcription, grammar_errors, num_pauses, filler_words, "phonims", "phonemes")
print(f"Grammar Score: {grammar_score}")
print(f"Fluency Score: {fluency_score}")
print(f"Pronunciation Score: {pronunciation_score}")

def generate_positive_feedback(grammar_score, fluency_score):
    feedback = {}

    # Grammar feedback
    if grammar_score >= 4.90:
        feedback['grammar'] = "Your grammar is outstanding, with exceptional clarity and no noticeable errors. Keep up the excellent work!"
    elif grammar_score >= 4.75:
        feedback['grammar'] = "Your grammar is excellent, with very few, nearly unnoticeable errors. A little more attention to detail will make it flawless."
    elif grammar_score >= 4.60:
        feedback['grammar'] = "Your grammar is strong, with only occasional minor errors. With consistent refinement, you can reach perfection."
    elif grammar_score >= 4.45:
        feedback['grammar'] = "Your grammar is very good, with a few minor areas for improvement. Regular practice will help you polish these aspects."
    elif grammar_score >= 4.30:
        feedback['grammar'] = "Your grammar is good and clear, though some noticeable errors exist. Focused practice can enhance your clarity even further."
    elif grammar_score >= 4.00:
        feedback['grammar'] = "Your grammar is adequate and generally understandable, but there is room for improvement in clarity and precision. Keep refining your skills."
    elif grammar_score >= 3.75:
        feedback['grammar'] = "Your grammar is improving, though noticeable errors are present. Continued practice and attention to detail will help you progress."
    elif grammar_score >= 3.50:
        feedback['grammar'] = "Your grammar shows potential, but frequent errors can distract from clarity. Focused effort on common patterns can significantly enhance your accuracy."
    else:
        feedback['grammar'] = "Your grammar is developing, but it requires consistent attention. Identify frequent errors and work on them to strengthen your communication skills."

    # Fluency feedback
    if fluency_score >= 4.90:
        feedback['fluency'] = "Your fluency is remarkable, with an effortless and natural flow. It is a pleasure to listen to!"
    elif fluency_score >= 4.75:
        feedback['fluency'] = "Your fluency is very strong, with rare and minor hesitations. Continued practice will make it even more seamless."
    elif fluency_score >= 4.60:
        feedback['fluency'] = "Your fluency is strong, with only slight hesitations. Practicing in real-time settings will make your speech even more fluid."
    elif fluency_score >= 4.45:
        feedback['fluency'] = "Your fluency is good, with occasional hesitations. Regular speaking exercises will help you improve your confidence and flow."
    elif fluency_score >= 4.30:
        feedback['fluency'] = "Your fluency is moderate, with room for improvement. Engaging in more speaking opportunities will help make your speech more natural."
    elif fluency_score >= 4.00:
        feedback['fluency'] = "Your fluency is adequate but could be smoother. Focused exercises in spontaneous speaking will enhance your confidence."
    elif fluency_score >= 3.75:
        feedback['fluency'] = "Your fluency is progressing, though noticeable pauses may affect the flow. Practicing continuous speech will help reduce hesitations."
    elif fluency_score >= 3.50:
        feedback['fluency'] = "Your fluency shows promise, but frequent hesitations disrupt the flow. Regular practice and confidence-building activities can help improve this."
    else:
        feedback['fluency'] = "Your fluency is developing, but consistent practice will greatly enhance your flow. Don’t hesitate to express yourself confidently!"

    # Pronunciation feedback
    if pronunciation_score >= 4.90:
        feedback['pronunciation'] = "Your pronunciation is exemplary, with clear and accurate articulation of every word. It’s a pleasure to hear!"
    elif pronunciation_score >= 4.75:
        feedback['pronunciation'] = "Your pronunciation is very strong, with only minor mispronunciations. A little more attention to detail will make it perfect."
    elif pronunciation_score >= 4.60:
        feedback['pronunciation'] = "Your pronunciation is strong, with occasional mispronunciations. Regular practice will help improve consistency."
    elif pronunciation_score >= 4.45:
        feedback['pronunciation'] = "Your pronunciation is good, but some sounds may be unclear. Focus on practicing difficult sounds to improve clarity."
    elif pronunciation_score >= 4.30:
        feedback['pronunciation'] = "Your pronunciation is moderate, with noticeable mispronunciations. Continued practice will improve your accuracy."
    elif pronunciation_score >= 4.00:
        feedback['pronunciation'] = "Your pronunciation is adequate but could use some improvement. Working on specific sounds will enhance your speech."
    elif pronunciation_score >= 3.75:
        feedback['pronunciation'] = "Your pronunciation is progressing, but some words are difficult to understand. Regular practice with pronunciation guides can help."
    elif pronunciation_score >= 3.50:
        feedback['pronunciation'] = "Your pronunciation shows potential, but frequent mispronunciations can hinder understanding. Focused practice will make a big difference."
    else:
        feedback['pronunciation'] = "Your pronunciation is still developing, but with consistent effort and guidance, you’ll see significant improvements."

    return feedback


feedback = generate_positive_feedback(grammar_score, fluency_score)

print("Feedback:")
print("Grammar:", feedback['grammar'])
print("Fluency:", feedback['fluency'])
print("Pronunciation:", feedback['pronunciation'])

Grammar Score: 4.17
Fluency Score: 5.0
Pronunciation Score: 4.25
Feedback:
Grammar: Your grammar is adequate and generally understandable, but there is room for improvement in clarity and precision. Keep refining your skills.
Fluency: Your fluency is remarkable, with an effortless and natural flow. It is a pleasure to listen to!
Pronunciation: Your pronunciation is adequate but could use some improvement. Working on specific sounds will enhance your speech.


In [28]:
candidate_info = {
    "grammar_score": {grammar_score},
    "fluency_score": {fluency_score},
    "pronunciation_score": {pronunciation_score},
    "candidate_response":{transcription},
    "fluency_feedback": {feedback['fluency']},
    "grammar_feedback": {feedback['grammar']},
    "pronunciation_feedback": {feedback['pronunciation']},
    "num_pauses": {num_pauses},
    "filler_words_count": {len(filler_words)},
    "words_per_second": {speaking_rate},
    "candidate_pronunciation": {phonemes_speaker}
}
prompt = f"""
You are an English language evaluation expert and a hiring manager. You will be provided with the assessment data and feedback from judges for a candidate on grammar, fluency, and pronunciation, along with the candidate's response. Your task is to evaluate the candidate's performance step-by-step, first presenting the judges' feedback as it is, and then explaining and elaborating on it using the provided scores, metrics, and response.

Follow these steps:

1. **Grammar Evaluation**:
   - **Feedback**: Present the feedback for grammar exactly as provided by the judges.
   - **Explanation and Elaboration**: Analyze the candidate's grammar based on the feedback, the score, and their response. Highlight specific strengths (e.g., accurate sentence structures) and weaknesses (e.g., grammatical errors like verb tense mismatches).

2. **Fluency Evaluation**:
   - **Feedback**: Present the feedback for fluency exactly as provided by the judges.
   - **Explanation and Elaboration**: Assess the candidate's fluency using the score and following metrics:
     - Number of filler words (e.g., "um," "uh," "like").
     - Number of noticeable pauses.
     - Words per second (WPS).
   - Provide an analysis of strengths (e.g., smooth delivery) and areas needing improvement (e.g., reducing filler words, increasing pace).

3. **Pronunciation Evaluation**:
   - **Feedback**: Present the feedback for pronunciation exactly as provided by the judges.
   - **Explanation and Elaboration**: Use the pronunciation score and the phones used by the candidate to evaluate clarity and articulation, highlighting specific strengths and weaknesses.

Finally, consolidate your analysis into a detailed final report that includes:
- The original judges' feedback for each section.
- The elaboration and explanation for each section.
- Specific, actionable recommendations for improvement.

Feedback from Judges:
- Grammar: {candidate_info['grammar_feedback']}
- Fluency: {candidate_info['fluency_feedback']}
- Pronunciation: {candidate_info['pronunciation_feedback']}
Assessment Scores:
- Grammar: {candidate_info['grammar_score']}/5
- Fluency: {candidate_info['fluency_score']}/5
- Pronunciation: {candidate_info['pronunciation_score']}/5

Additional Metrics:
- Fluency:
  - Number of Filler Words: {candidate_info['filler_words_count']}
  - Number of Pauses: {candidate_info['num_pauses']}
  - Words per Second (WPS): {candidate_info['words_per_second']}
- Pronunciation:
  - Candidate's Pronunciation: {candidate_info['candidate_pronunciation']}

Candidate's Response:
{candidate_info['candidate_response']}

Generate the final report with the structure specified.
"""


response = language_model.generate_content(prompt)
print("Generated Report:")
print(response.text)



Generated Report:
## Candidate Language Evaluation Report

### Grammar Evaluation

**Judges' Feedback**: "Your grammar is adequate and generally understandable, but there is room for improvement in clarity and precision. Keep refining your skills."

**Explanation and Elaboration**:
The candidate's grammar score of 4.17 indicates that they generally use correct sentence structures and tenses. However, there is room for improvement in clarity and precision. Some specific areas for improvement could include:
- Eliminating minor grammatical errors, such as verb tense mismatches (e.g., "I was going yesterday" instead of "I went yesterday").
- Using more specific and precise language to enhance clarity and avoid ambiguity.
- Proofreading written responses carefully to identify and correct any grammatical errors.

### Fluency Evaluation

**Judges' Feedback**: "Your fluency is remarkable, with an effortless and natural flow. It is a pleasure to listen to!"

**Explanation and Elaboration**:
The